In [1]:
import sympy

In [4]:
# Параметры программы
links_count = 2
calculate_inertia = False #True

# Константы
g = sympy.Symbol('g') # Ускорение свободного падения

# Параметры задачи
## Длина каждого звена
l = sympy.IndexedBase('l')

## Масса каждого звена 
m = sympy.IndexedBase('m')

## Моменты инерции каждого звена
if calculate_inertia:
    I = [
        m[i] * l[i] * l[i] / 3
        for i in range(0, links_count)
    ]
else:
    I = sympy.IndexedBase('I')

$$
    u = M(q)\ddot q + L(q, \dot q)
$$

Тогда в силу $M(q) > 0$ имеем

$$
    \ddot q = M^{-1}(q)(u - L(q, \dot q))
$$

In [28]:
# Фазовые переменные
t = sympy.symbols('t')
q = [sympy.Function('q_'+str(i))(t) for i in range(0, links_count)]

dot_q = [
    sympy.diff(q[i], t)
    for i in range(0, links_count)
]

# Положение центра каждого звена
x_center = [
    sum([l[j] * sympy.cos(q[j]) for j in range(0, i)]) + l[i]/2*sympy.cos(q[i])
    for i in range(0, links_count)
]

y_center = [
    sum([l[j]*sympy.sin(q[j]) for j in range(0, i)]) + l[i]/2*sympy.sin(q[i])
    for i in range(0, links_count)
]

# Скорость центра каждого звена в квадрате
v_square = [
    sympy.diff(x_center[i], t)**2 + sympy.diff(y_center[i], t)**2
    for i in range(0, links_count)
]

# Скорость вращения каждого звена
omega = [
    dot_q[i]
    for i in range(0, links_count)
]

# Общая кинетическая энегия системы
K = sum([
    m[i] * v_square[i]/2 + I[i] * omega[i] * omega[i] / 2 
    for i in range(0, links_count)
])

# Общая потенциальная энергия системы
Pi = sum([
    m[i] * g * y_center[i]
    for i in range(0, links_count)
])

# Лангражиан
L = K - Pi

# Моменты внешних сил
tau = [
    sympy.simplify(sympy.diff(sympy.diff(L, dot_q[i]), t) - sympy.diff(L, q[i]))
    for i in range(0, links_count)
]

tau_vec = sympy.matrices.Matrix([tau[i] for i in range(links_count)])

sympy.simplify(tau_vec)

Matrix([
[g*cos(q_0(t))*l[0]*m[0]/2 + g*cos(q_0(t))*l[0]*m[1] + sin(q_0(t) - q_1(t))*Derivative(q_1(t), t)**2*l[0]*l[1]*m[1]/2 + cos(q_0(t) - q_1(t))*Derivative(q_1(t), (t, 2))*l[0]*l[1]*m[1]/2 + Derivative(q_0(t), (t, 2))*I[0] + Derivative(q_0(t), (t, 2))*l[0]**2*m[0]/4 + Derivative(q_0(t), (t, 2))*l[0]**2*m[1]],
[                                                                    g*cos(q_1(t))*l[1]*m[1]/2 - sin(q_0(t) - q_1(t))*Derivative(q_0(t), t)**2*l[0]*l[1]*m[1]/2 + cos(q_0(t) - q_1(t))*Derivative(q_0(t), (t, 2))*l[0]*l[1]*m[1]/2 + Derivative(q_1(t), (t, 2))*I[1] + Derivative(q_1(t), (t, 2))*l[1]**2*m[1]/4]])

In [29]:
z = sympy.IndexedBase('z')
dot_z = sympy.IndexedBase('\dot z')

for i in range(links_count):
    for j in range(links_count):
        tau_vec[i] = tau_vec[i].subs(sympy.diff(q[j], (t,2)), dot_z[links_count + j])
        tau_vec[i] = tau_vec[i].subs(sympy.diff(q[j], t), z[links_count + j])
        tau_vec[i] = tau_vec[i].subs(q[j], z[j])

sympy.simplify(tau_vec)

Matrix([
[g*cos(z[0])*l[0]*m[0]/2 + g*cos(z[0])*l[0]*m[1] + sin(z[0] - z[1])*l[0]*l[1]*m[1]*z[3]**2/2 + cos(z[0] - z[1])*\dot z[3]*l[0]*l[1]*m[1]/2 + I[0]*\dot z[2] + \dot z[2]*l[0]**2*m[0]/4 + \dot z[2]*l[0]**2*m[1]],
[                                                 g*cos(z[1])*l[1]*m[1]/2 - sin(z[0] - z[1])*l[0]*l[1]*m[1]*z[2]**2/2 + cos(z[0] - z[1])*\dot z[2]*l[0]*l[1]*m[1]/2 + I[1]*\dot z[3] + \dot z[3]*l[1]**2*m[1]/4]])

In [43]:
M = sympy.matrices.Matrix([
    [
        sympy.collect(sympy.simplify(tau_vec[i]).expand(), dot_z[links_count+j]).coeff(dot_z[links_count+j], 1)
        for j in range(links_count)
    ]
    for i in range(links_count)
])

M

Matrix([
[I[0] + l[0]**2*m[0]/4 + l[0]**2*m[1], cos(z[0] - z[1])*l[0]*l[1]*m[1]/2],
[   cos(z[0] - z[1])*l[0]*l[1]*m[1]/2,             I[1] + l[1]**2*m[1]/4]])

In [47]:
def remove_order(expr):
    for i in range(links_count):
        expr = sympy.simplify(expr).expand()
        expr = sympy.collect(expr, dot_z[links_count+i]).coeff(dot_z[links_count+i], 0)
    return expr

L = sympy.matrices.Matrix([
    remove_order(tau_vec[i])
    for i in range(links_count)
])

L

Matrix([
[g*cos(z[0])*l[0]*m[0]/2 + g*cos(z[0])*l[0]*m[1] + sin(z[0] - z[1])*l[0]*l[1]*m[1]*z[3]**2/2],
[                        g*cos(z[1])*l[1]*m[1]/2 - sin(z[0] - z[1])*l[0]*l[1]*m[1]*z[2]**2/2]])

In [50]:
M_inv = sympy.simplify(M.inv(method='LU'))
M_inv

Matrix([
[        (16*I[1] + 4*l[1]**2*m[1])/((4*I[1] + l[1]**2*m[1])*(4*I[0] + l[0]**2*m[0] + 4*l[0]**2*m[1]) - 4*cos(z[0] - z[1])**2*l[0]**2*l[1]**2*m[1]**2),           -8*cos(z[0] - z[1])*l[0]*l[1]*m[1]/((4*I[1] + l[1]**2*m[1])*(4*I[0] + l[0]**2*m[0] + 4*l[0]**2*m[1]) - 4*cos(z[0] - z[1])**2*l[0]**2*l[1]**2*m[1]**2)],
[-8*cos(z[0] - z[1])*l[0]*l[1]*m[1]/((4*I[1] + l[1]**2*m[1])*(4*I[0] + l[0]**2*m[0] + 4*l[0]**2*m[1]) - 4*cos(z[0] - z[1])**2*l[0]**2*l[1]**2*m[1]**2), (16*I[0] + 4*l[0]**2*m[0] + 16*l[0]**2*m[1])/((4*I[1] + l[1]**2*m[1])*(4*I[0] + l[0]**2*m[0] + 4*l[0]**2*m[1]) - 4*cos(z[0] - z[1])**2*l[0]**2*l[1]**2*m[1]**2)]])

$$
    \dot z = f(z, u)
$$

In [57]:
u = sympy.IndexedBase('u')

u_vec = sympy.Matrix([
    u[i]
    for i in range(links_count)
])

f = sympy.Matrix([z[links_count + i] for i in range(links_count)])
f = f.col_join(M_inv * (u_vec - L))

f

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      z[2]],
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      z[3]],
[  

$$
    \dot z = a(x) + b(x)u
$$

In [60]:
a = sympy.Matrix([z[links_count + i] for i in range(links_count)])
a = a.col_join(sympy.simplify(- M_inv * L))
a

Matrix([
[                                                                                                                                                                                                                                                                                                                              z[2]],
[                                                                                                                                                                                                                                                                                                                              z[3]],
[               2*(2*(g*cos(z[1]) - sin(z[0] - z[1])*l[0]*z[2]**2)*cos(z[0] - z[1])*l[1]**2*m[1]**2 - (4*I[1] + l[1]**2*m[1])*(g*cos(z[0])*m[0] + 2*g*cos(z[0])*m[1] + sin(z[0] - z[1])*l[1]*m[1]*z[3]**2))*l[0]/((4*I[1] + l[1]**2*m[1])*(4*I[0] + l[0]**2*m[0] + 4*l[0]**2*m[1]) - 4*cos(z[0] - z[1])**2*l[0]**2*l[1]**2*m[1]**2)],
[2*(-(g*cos(z

In [61]:
b = sympy.zeros(links_count, links_count)

b = b.col_join(M_inv)
b

Matrix([
[                                                                                                                                                    0,                                                                                                                                                               0],
[                                                                                                                                                    0,                                                                                                                                                               0],
[        (16*I[1] + 4*l[1]**2*m[1])/((4*I[1] + l[1]**2*m[1])*(4*I[0] + l[0]**2*m[0] + 4*l[0]**2*m[1]) - 4*cos(z[0] - z[1])**2*l[0]**2*l[1]**2*m[1]**2),           -8*cos(z[0] - z[1])*l[0]*l[1]*m[1]/((4*I[1] + l[1]**2*m[1])*(4*I[0] + l[0]**2*m[0] + 4*l[0]**2*m[1]) - 4*cos(z[0] - z[1])**2*l[0]**2*l[1]**2*m[1]**2)],
[-8*cos(z[0] - z[1])*l[0]*l[1]*m[1]/((4*I[1] + l[